In [ ]:
import math

# Function to read header of .bmp image
def read_bitmap_header(file_path):
    with open(file_path, 'rb') as file:
        # Read the BMP header
        header_data = file.read(54)

        # Extract information from the header
        signature = header_data[:2].decode('utf-8')
        file_size = int.from_bytes(header_data[2:6], byteorder='little')
        reserved = int.from_bytes(header_data[6:10], byteorder='little')
        data_offset = int.from_bytes(header_data[10:14], byteorder='little')
        header_size = int.from_bytes(header_data[14:18], byteorder='little')
        width = int.from_bytes(header_data[18:22], byteorder='little')
        height = int.from_bytes(header_data[22:26], byteorder='little')
        color_planes = int.from_bytes(header_data[26:28], byteorder='little')
        bit_depth = int.from_bytes(header_data[28:30], byteorder='little')
        compression = int.from_bytes(header_data[30:34], byteorder='little')
        image_size = int.from_bytes(header_data[34:38], byteorder='little')
        horizontal_resolution = int.from_bytes(header_data[38:42], byteorder='little')
        vertical_resolution = int.from_bytes(header_data[42:46], byteorder='little')
        colors_in_palette = int.from_bytes(header_data[46:50], byteorder='little')
        important_colors = int.from_bytes(header_data[50:54], byteorder='little')

        # Print extracted header information
        print("Signature:", signature)
        print("File Size:", file_size)
        print("Reserved:", reserved)
        print("Data Offset:", data_offset)
        print("Header Size:", header_size)
        print("Width:", width)
        print("Height:", height)
        print("Color Planes:", color_planes)
        print("Bits per pixel:", bit_depth)
        print("Compression:", compression)
        print("Image Size:", image_size)
        print("Horizontal Resolution:", horizontal_resolution)
        print("Vertical Resolution:", vertical_resolution)
        print("Colors in Palette:", colors_in_palette)
        print("Important Colors:", important_colors)
        return header_data

# Function to read color palette of .bmp image
def read_bitmap_color_palette(file_path):
    with open(file_path, 'rb') as file:
        # Read the BMP header (assuming 54 bytes header size)
        header_data = file.read(54)
        color_palette = []
        bytes_per_pixel = int((int.from_bytes(header_data[28:30], byteorder='little')) / 8)
        if bytes_per_pixel == 1:
            for _ in range(256):
                blue = int.from_bytes(file.read(1), byteorder='little')
                green = int.from_bytes(file.read(1), byteorder='little')
                red = int.from_bytes(file.read(1), byteorder='little')
                padding_byte = file.read(1)  # Skip padding byte
                color_palette.append((blue, green, red))

        return color_palette

# Function to read the pixel information of .bmp image
def read_bitmap_pixel_data(file_path):
    with open(file_path, 'rb') as file:
        # Read the BMP header (assuming 54 bytes header size)
        header_data = file.read(54)

        # Extract image dimensions from the header
        width = int.from_bytes(header_data[18:22], byteorder='little')
        height = int.from_bytes(header_data[22:26], byteorder='little')

        # Calculate padding bytes per row
        bytes_per_pixel = int((int.from_bytes(header_data[28:30], byteorder='little')) / 8)
        padding = (4 - (width * bytes_per_pixel) % 4) % 4
        pixel_data = []

        color_palette = []
        if bytes_per_pixel == 1:
            for _ in range(256):
                blue = int.from_bytes(file.read(1), byteorder='little')
                green = int.from_bytes(file.read(1), byteorder='little')
                red = int.from_bytes(file.read(1), byteorder='little')
                _ = file.read(1)  # Skip padding byte
                color_palette.append((blue, green, red))

        # Read pixel data row by row
        for h in range(height):
            row_data = []
            for w in range(width):
                if bytes_per_pixel == 3:
                    blue = int.from_bytes(file.read(1), byteorder='little')
                    green = int.from_bytes(file.read(1), byteorder='little')
                    red = int.from_bytes(file.read(1), byteorder='little')
                    row_data.append((blue, green, red))
                if bytes_per_pixel == 1:
                    gray = int.from_bytes(file.read(1), byteorder='little')
                    row_data.append(gray)
            file.read(padding)  # Skip padding bytes
            pixel_data.append(row_data)
        return pixel_data

# Function to write .bmp image using header data and pixel data
def write_bitmap_file(header_data, color_palette, pixel_data, filename):
    # Extract image dimensions from the header
    width = int.from_bytes(header_data[18:22], byteorder='little')
    height = int.from_bytes(header_data[22:26], byteorder='little')
    bytes_per_pixel = int((int.from_bytes(header_data[28:30], byteorder='little')) / 8)
    padding = (4 - (width * bytes_per_pixel) % 4) % 4
    zero = 0

    with open(filename, 'wb') as file:
        file.write(header_data)
        if bytes_per_pixel == 1:
            for i in range(256):
                file.write(color_palette[i][0].to_bytes(1, "little"))
                file.write(color_palette[i][1].to_bytes(1, "little"))
                file.write(color_palette[i][2].to_bytes(1, "little"))
                file.write(zero.to_bytes(1, "little"))
        for h in range(height):
            for w in range(width):
                if bytes_per_pixel == 3:
                    file.write(pixel_data[h][w][0].to_bytes(1, "little"))
                    file.write(pixel_data[h][w][1].to_bytes(1, "little"))
                    file.write(pixel_data[h][w][2].to_bytes(1, "little"))
                if bytes_per_pixel == 1:
                    file.write(pixel_data[h][w].to_bytes(1, "little"))
            for p in range(padding):
                file.write(zero.to_bytes(1, "little"))

# Function to scale up or scale down an image
def scale_image(header_data, color_palette, pixel_data, filename, scale_factor):
    width = int.from_bytes(header_data[18:22], byteorder='little')
    height = int.from_bytes(header_data[22:26], byteorder='little')
    new_height = int(height * scale_factor)
    new_width = int(width * scale_factor)
    bytes_per_pixel = int((int.from_bytes(header_data[28:30], byteorder='little')) / 8)
    padding = (4 - (width * bytes_per_pixel) % 4) % 4
    scaled_image = [[0] * new_width for _ in range(new_height)]
    zero = 0

    for y in range(new_height):
        for x in range(new_width):
            original_x = int(x / scale_factor)
            original_y = int(y / scale_factor)
            scaled_image[y][x] = pixel_data[original_y][original_x]

    with open(filename, 'wb') as file:
        for i in range(54):
            if i >= 18 and i < 26:
                if i == 18:
                    file.write(new_width.to_bytes(4, "little"))
                if i == 22:
                    file.write(new_height.to_bytes(4, "little"))
                continue
            file.write(header_data[i].to_bytes(1, "little"))

        # Writing color palette
        if bytes_per_pixel == 1:
            for i in range(256):
                file.write(color_palette[i][0].to_bytes(1, "little"))
                file.write(color_palette[i][1].to_bytes(1, "little"))
                file.write(color_palette[i][2].to_bytes(1, "little"))
                file.write(zero.to_bytes(1, "little"))

        # Writing pixel data
        for h in range(new_height):
            for w in range(new_width):
                if bytes_per_pixel == 3:
                    file.write(scaled_image[h][w][0].to_bytes(1, "little"))
                    file.write(scaled_image[h][w][1].to_bytes(1, "little"))
                    file.write(scaled_image[h][w][2].to_bytes(1, "little"))
                if bytes_per_pixel == 1:
                    file.write(scaled_image[h][w].to_bytes(1, "little"))
            for p in range(padding):
                file.write(zero.to_bytes(1, "little"))

# Function to rotate an image
def rotate_image(header_data, color_palette, pixel_data, filename, angle):
    width = int.from_bytes(header_data[18:22], byteorder='little')
    height = int.from_bytes(header_data[22:26], byteorder='little')
    bytes_per_pixel = int((int.from_bytes(header_data[28:30], byteorder='little')) / 8)
    zero = 0
    padding = (4 - (width * bytes_per_pixel) % 4) % 4
    # Convert the angle to radians
    angle_rad = math.radians(angle)

    # Calculate the new dimensions of the rotated image
    new_width = int(abs(width * math.cos(angle_rad)) + abs(height * math.sin(angle_rad)))
    new_height = int(abs(width * math.sin(angle_rad)) + abs(height * math.cos(angle_rad)))
    new_height = (int)((new_height - 1) / 4) * 4 + 4
    new_width = (int)((new_width - 1) / 4) * 4 + 4
    # Initialize the rotated image array with zeros
    # rotated_img= [[0 for _ in range(new_width)] for _ in range(new_height)]
    if bytes_per_pixel == 1:
        rotated_img = [[0] * new_width for _ in range(new_height)]
    if bytes_per_pixel == 3:
        rotated_img = [[[0] * 3 for _ in range(new_width)] for _ in range(new_height)]

    # Calculate the center point of the original image
    xcenter_old = width // 2
    ycenter_old = height // 2

    # Calculate the center point of the rotated image
    xcenter = new_width // 2
    ycenter = new_height // 2

    # Rotate the image pixel by pixel
    for y in range(new_height):
        for x in range(new_width):
            # Calculate the corresponding pixel position in the original image
            orig_x = int(((x - xcenter) * math.cos(angle_rad) + (y - ycenter) * math.sin(angle_rad)) + xcenter_old)
            orig_y = int((-(x - xcenter) * math.sin(angle_rad) + (y - ycenter) * math.cos(angle_rad)) + ycenter_old)
            if x == 359 and y == 180:
                print(orig_x, orig_y)

            # Check if the calculated position is within the original image bounds
            if 0 <= orig_x < width and 0 <= orig_y < height:
                if bytes_per_pixel == 1:
                    rotated_img[y][x] = pixel_data[orig_y][orig_x]
                if bytes_per_pixel == 3:
                    rotated_img[y][x][0] = pixel_data[orig_y][orig_x][0]
                    rotated_img[y][x][1] = pixel_data[orig_y][orig_x][1]
                    rotated_img[y][x][2] = pixel_data[orig_y][orig_x][2]

    with open(filename, 'wb') as file:
        # f.write(header_data)
        for i in range(54):
            if i >= 18 and i < 26:
                if i == 18:
                    file.write(new_width.to_bytes(4, "little"))
                if i == 22:
                    file.write(new_height.to_bytes(4, "little"))
                continue
            file.write(header_data[i].to_bytes(1, "little"))

        # Writing color palette
        if bytes_per_pixel == 1:
            for i in range(256):
                file.write(color_palette[i][0].to_bytes(1, "little"))
                file.write(color_palette[i][1].to_bytes(1, "little"))
                file.write(color_palette[i][2].to_bytes(1, "little"))
                file.write(zero.to_bytes(1, "little"))

        # Writing pixel data
        for h in range(new_height):
            for w in range(new_width):
                if bytes_per_pixel == 3:
                    file.write(rotated_img[h][w][0].to_bytes(1, "little"))
                    file.write(rotated_img[h][w][1].to_bytes(1, "little"))
                    file.write(rotated_img[h][w][2].to_bytes(1, "little"))
                if bytes_per_pixel == 1:
                    file.write(rotated_img[h][w].to_bytes(1, "little"))
            for p in range(padding):
                file.write(zero.to_bytes(1, "little"))



In [ ]:
# Running the functions on images
file_path = r"/content/corn.bmp"
header_data = read_bitmap_header(file_path)
color_palette = read_bitmap_color_palette(file_path)
pixel_data = read_bitmap_pixel_data(file_path)

scale_image(header_data, color_palette, pixel_data, "corn_scaledown.bmp", 0.5)
scale_image(header_data, color_palette, pixel_data, "corn_scaleup.bmp", 2)
rotate_image(header_data, color_palette, pixel_data, "corn_rotate90.bmp", 90)
rotate_image(header_data, color_palette, pixel_data, "corn_rotate45.bmp", 45)

file_path = r"/content/lena_colored_256.bmp"
header_data = read_bitmap_header(file_path)
color_palette = read_bitmap_color_palette(file_path)
pixel_data = read_bitmap_pixel_data(file_path)

scale_image(header_data, color_palette, pixel_data, "lena_colored_256_scaledown.bmp", 0.5)
scale_image(header_data, color_palette, pixel_data, "lena_colored_256_scaleup.bmp", 2)
rotate_image(header_data, color_palette, pixel_data, "lena_colored_256_rotate90.bmp", 90)
rotate_image(header_data, color_palette, pixel_data, "lena_colored_256_rotate45.bmp", 45)

file_path = r"/content/cameraman.bmp"
header_data = read_bitmap_header(file_path)
color_palette = read_bitmap_color_palette(file_path)
pixel_data = read_bitmap_pixel_data(file_path)

scale_image(header_data, color_palette, pixel_data, "cameraman_scaledown.bmp", 0.5)
scale_image(header_data, color_palette, pixel_data, "cameraman_scaleup.bmp", 2)
rotate_image(header_data, color_palette, pixel_data, "cameraman_rotate90.bmp", 90)
rotate_image(header_data, color_palette, pixel_data, "cameraman_rotate45.bmp", 45)

Signature: BM
File Size: 130558
Reserved: 0
Data Offset: 1078
Header Size: 40
Width: 312
Height: 415
Color Planes: 1
Bits per pixel: 8
Compression: 0
Image Size: 129480
Horizontal Resolution: 0
Vertical Resolution: 0
Colors in Palette: 256
Important Colors: 0
180 56
172 80
Signature: BM
File Size: 196662
Reserved: 0
Data Offset: 54
Header Size: 40
Width: 256
Height: 256
Color Planes: 1
Bits per pixel: 24
Compression: 0
Image Size: 196608
Horizontal Resolution: 3780
Vertical Resolution: 3780
Colors in Palette: 0
Important Colors: 0
251 1
Signature: BM
File Size: 66614
Reserved: 0
Data Offset: 1078
Header Size: 40
Width: 256
Height: 256
Color Planes: 1
Bits per pixel: 8
Compression: 0
Image Size: 65536
Horizontal Resolution: 0
Vertical Resolution: 0
Colors in Palette: 256
Important Colors: 0
251 1
